In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::142401413602:role/service-role/AmazonSageMaker-ExecutionRole-20220518T150261
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: c8e69144-c9a6-4c59-8bed-b6ee923f4aee
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
Waiting for session c8e69144-c9a6-4c59-8bed-b6ee923f4aee to get into ready status...
Session c8e69144-c9a6-4c59-8bed-b6ee923f4aee has been created.



In [2]:
spark = SparkSession.builder.appName("example").getOrCreate()

In [16]:
df = spark.sql(
"""

	select 
	c.cpf_customer
	, count(distinct c.id_issue) as contatos_12m
	, count(distinct case when dt_chat >= date_add(date_trunc('month', current_date), -180 ) then c.id_issue end) as contatos_6m
	, max(is_resolvido_bot) as teve_alguma_resolucao_bot
	, max(is_finalizacao_automacao) as teve_finalizacao_automatica
	, max(nr_tpr) as max_tpr
	, max(nr_tmr) as max_tmr
	, max(nr_tma) as max_tma
	, max(nr_tma_resolucao) as max_tma_resolucao
	, max(nr_tmat) as max_tr --tr
	, min(nr_csat) as min_csat
    , max(dt_chat) as recencia_contato
	FROM cx_curated_zone.cx_contact_rate c
	left join cx_curated_zone.helpshift_issues AS issues_macro ON c.id_issue = issues_macro.id_issue
	left join cx_curated_zone.helpshift_tag_niveis AS helpshift_tag ON c.id_issue = helpshift_tag.id_issue
	where dt_chat >= date_add(date_trunc('month', current_date), -360 ) and dt_chat < date_trunc('month', current_date) 
	group by 1

"""
)


In [17]:
df.show(5)

+------------+------------+-----------+-------------------------+---------------------------+-------+-------+-------+-----------------+------+--------+----------------+
|cpf_customer|contatos_12m|contatos_6m|teve_alguma_resolucao_bot|teve_finalizacao_automatica|max_tpr|max_tmr|max_tma|max_tma_resolucao|max_tr|min_csat|recencia_contato|
+------------+------------+-----------+-------------------------+---------------------------+-------+-------+-------+-----------------+------+--------+----------------+
| 11221029924|           9|          5|                        1|                          1|    588|    117|   2880|                9|  2888|       5|      2023-06-10|
| 72986441572|           3|          2|                        1|                          1|      6|      0|    732|               80|   738|    null|      2023-03-02|
| 06549938589|           1|          0|                        0|                          0|      2|      0|   3227|             3227|  3229|    null|    

In [13]:
df.head(1)

[Row(cpf_customer='88583279691', contatos_12m=1, contatos_6m=0, teve_alguma_resolucao_bot=0, teve_alguma_resolucao_bot=1, max_tpr=1290, max_tmr=27, max_tma=722, max_tma_resolucao=2, max_tr=2012, min_csat=None, recencia_contato=datetime.date(2022, 8, 6))]


In [18]:
df.count()

2111732


In [24]:
df.write.mode("overwrite").parquet("s3a://data-workspace-will-prod/flavia-costa/base_agg_atendimento")